In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
# from resources.plotcm import plot_confusion_matrix

import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm

from pytorch_datagen import DataGen
from pytorch_datagen import *
from resunetPlusPlus_pytorch_copy import build_resunetplusplus

In [2]:
#for name, param in model.named_parameters():
#    print("name: ", name, " param: ", param, " requires_grad: ", param.requires_grad)

In [30]:
def mask_to_3d(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

if __name__ == "__main__":
    model_path = "ColonoscopyTrained_resUnetPlusPlus.pkl"
    save_path = "result"
    test_path = "../new_data/kvasir_segmentation_dataset/test/"
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("GPU available: ", torch.cuda.is_available())

    image_size = 256
    batch_size = 1

    test_image_paths = glob(os.path.join(test_path, "images", "*"))
    test_mask_paths = glob(os.path.join(test_path, "masks", "*"))
    test_image_paths.sort()
    test_mask_paths.sort()

    ## Create result folder
    try:
        os.mkdir(save_path)
    except:
        pass

    ## Model
    model = build_resunetplusplus()
    model.load_state_dict(torch.load(model_path))
    model.to(device)
      
    test_steps = len(test_image_paths)//batch_size
    print("test steps: ", test_steps)
    test_gen = DataGen(image_size, test_image_paths, test_mask_paths)
    test_loader = torch.utils.data.DataLoader(test_gen)
    
    #switch off autograd
    with torch.no_grad():
        #det the model in evaluation mode
        model.eval()
        for batch in test_loader:
            images, masks = batch
            
            images = images.to(device, dtype=torch.float)
            masks = masks.to(device, dtype=torch.float)
            print("here: ", images.size())
            
            images = images.unsqueeze(1).to(device)
            print("here 2: ", images.size())
            masks = masks.permute(0, 3, 1, 2).to(device)
            
            predict_mask = model(images).to(device)
            
            loss = F.mse_loss(predict_mask, masks).to(device)
            
            
            predict_mask = (predict_mask > 0.5) * 255.0
            
            sep_line = np.ones((image_size, 10, 3)) * 255
            
            mask = mask_to_3d(masks)
            predict_mask = mask_to_3d(predict_mask)
            
            all_images = [image * 255, sep_line, mask * 255, sep_line, predict_mask]
            cv2.imwrite(f"{save_path}/{i}.png", np.concatenate(all_images, axis=1))
    print("done")


GPU available:  True
test steps:  100
here:  torch.Size([1, 256, 256])
here 2:  torch.Size([1, 1, 256, 256])


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [10]:
type(test_gen)

pytorch_datagen.DataGen

In [30]:
test_path = "new_data/kvasir_segmentation_dataset/test/"
test_image_paths = glob(os.path.join(test_path, "images", "*"))
test_image_paths.sort()
test_steps = len(test_image_paths)
print(test_steps)

0
